In [1]:
import pandas as pd
import networkx as nx

superheropowers=pd.read_csv('/content/drive/MyDrive/marvelproject/superheroes_power_matrix.csv')
heronet=pd.read_csv('/content/drive/MyDrive/marvelproject/hero-network.csv')
alignment=pd.read_csv('/content/drive/MyDrive/marvelproject/marvel_characters_info.csv')

In [2]:
#construct a small dataframe with just hero name and alignment and capitalize
alignment=alignment[['Name', 'Alignment']]
alignment['Name']=[x.upper() for x in alignment['Name']]
superheropowers['Name']=[x.upper() for x in superheropowers['Name']]

#clean the heronet dataframe
heronet['hero1'] = heronet['hero1'].str.split('/').str[0]
heronet['hero2'] = heronet['hero2'].str.split('/').str[0]

#get a Marvel only dataset. we can do this because our hero net is only
#marvel characters 
hero1=set(heronet['hero1'])
hero2=set(heronet['hero2'])
heroset=list(hero1.union(hero2))
marvelchars=superheropowers[superheropowers['Name'].isin(heroset)]

#merge our dataframes and set our alighment to good guys
marvelchars=marvelchars.reset_index(drop=True)
marvelheros=marvelchars.merge(alignment, how='inner', left_on='Name', right_on='Name')
goodguys=marvelheros[marvelheros['Alignment']=='good']
goodguys=goodguys.drop_duplicates()
goodguys=goodguys.reset_index(drop=True)

<ipython-input-2-04a158f0afc4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alignment['Name']=[x.upper() for x in alignment['Name']]


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
#we drop unnecessary columns for training purposes and reset our index
trainer=goodguys.drop(['Name', 'Alignment'], axis=1)
trainer=trainer.reset_index(drop=True)
cosine_sim = cosine_similarity(trainer)

In [4]:
#we are only concerned with the network of our avengers team so we center on their social networks
#hawkeye is listed as hawk in this dataset
avengers=['captain america', 'ant-man', 'iron man', 'black widow', 'thor', 'spider-man', 'hulk', 'hawk', 'wasp', 'Wolverine']
avengers=[x.upper() for x in avengers]
avengernet = heronet[(heronet['hero1'].isin(avengers)) | (heronet['hero2'].isin(avengers))]
G=nx.from_pandas_edgelist(avengernet, source='hero1', target='hero2')

In [5]:
#we get our index for SPIDER-MAN
index1=goodguys.index[goodguys['Name']=='SPIDER-MAN'].tolist()[0]
#we calculate the cosine similarities of other heroes with SPIDER-MAN and sort
similar_char = list(enumerate(cosine_sim[index1]))
similar_char.sort(key=lambda x:x[1], reverse= True)
#we don't need SPIDER-MAN to compare with himself so we skip him.
sorted_score = similar_char[1:]

#Now we construct a dataframe with the heroes and their similarity scores
potentials=[]
score=[]
i = 0
for item in sorted_score:
  cosscore=item[1]
  char_name=goodguys[goodguys.index==item[0]]['Name'].values[0]
  potentials.append(char_name)
  score.append(cosscore)
#we will only center on the 10 highest similarity scores
  i = i+1
  if i > 10:
      break
dfconstruct=pd.DataFrame({'potential name':potentials, 'cos_score':score})

In [6]:
#calculate centrality and put into a dataframe
centralities1=nx.closeness_centrality(G)
centdf=pd.DataFrame(centralities1.items(), columns=['Name', 'Closeness_Centrality'])
#ensure the recommendations from centralities are in our list of similar heroes
recommended1=centdf[centdf['Name'].isin(potentials)]
#we also want to exclude our avengers already on the team
recommended1=recommended1[~recommended1['Name'].isin(avengers)]
#construct a sorted dataframe and join it onto our similarities dataframe
sortedrecs=recommended1.sort_values(by='Closeness_Centrality', ascending=False)
recommendationdf=sortedrecs.merge(dfconstruct, how='inner', left_on='Name', right_on='potential name')
#create an aggregate score and drop unnecessary columns
recommendationdf['comboscore']=recommendationdf['Closeness_Centrality']*recommendationdf['cos_score']
recommendationdf=recommendationdf.sort_values(by='comboscore', ascending=False)
recommendationdf=recommendationdf[['Name', 'Closeness_Centrality', 'cos_score', 'comboscore']]

In [7]:
recommendationdf

,Name,Closeness_Centrality,cos_score,comboscore
0,BEAST,0.500628,0.717137,0.359019
3,GAMORA,0.500628,0.620174,0.310476
6,FERAL,0.456146,0.670820,0.305992
7,ARACHNE,0.439124,0.645497,0.283453
1,CAPTAIN BRITAIN,0.500628,0.565685,0.283198
2,SASQUATCH,0.500628,0.547723,0.274205
4,SHANG-CHI,0.495036,0.547723,0.271142
8,HYBRID,0.376498,0.715097,0.269233
5,BATTLESTAR,0.485989,0.547723,0.266187
